# MAPS Pandas, utilisation

In [54]:
import pandas as pd

data = pd.read_csv("https://bit.ly/felonies-dataset")
data['DoA'] = pd.to_datetime(data['Date of Arrest'])
data.dtypes

Date of Arrest            object
Age                        int64
Convicted                 object
DoA               datetime64[ns]
dtype: object

## Destruction Columns

In [55]:
data.drop(['Date of Arrest'], axis='columns', inplace = True)
data.head()

,Age,Convicted,DoA
0,77,Yes,2014-07-16 14:59:18
1,28,Yes,2002-12-19 22:17:59
2,21,No,1994-06-28 09:31:40
3,45,Yes,1985-05-14 09:52:17
4,39,Yes,1985-05-25 14:06:28


In [56]:
dow2weektime = {0: 'Debut', 1: 'Debut',
               2: 'Millieu', 3: 'Millieu', 4: 'Millieu',
               5: 'Weekend', 6: 'Weekend'}
data.DoA.dt.dayofweek.map(dow2weektime)

0      Millieu
1      Millieu
2        Debut
3        Debut
4      Weekend
        ...   
995      Debut
996      Debut
997      Debut
998      Debut
999    Weekend
Name: DoA, Length: 1000, dtype: object

In [57]:
id2str = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
str2id = {value: idx for idx, value in enumerate(id2str)}
str2id

{'monday': 0,
 'tuesday': 1,
 'wednesday': 2,
 'thursday': 3,
 'friday': 4,
 'saturday': 5,
 'sunday': 6}

In [58]:
id2str = {idx: value for idx, value in enumerate(str2id)}
id2str

{0: 'monday',
 1: 'tuesday',
 2: 'wednesday',
 3: 'thursday',
 4: 'friday',
 5: 'saturday',
 6: 'sunday'}

In [59]:
data['DoA'].dt.dayofweek.map(id2str)

0      wednesday
1       thursday
2        tuesday
3        tuesday
4       saturday
         ...    
995      tuesday
996       monday
997      tuesday
998       monday
999       sunday
Name: DoA, Length: 1000, dtype: object

In [60]:
periodeSemaine = {'monday': 'debut',
                 'tuesday': 'debut',
                 'wednesday': 'millieu',
                 'thursday': 'millieu',
                 'friday': 'millieu',
                 'saturday': 'week-end',
                 'sunday': 'week-end'}

In [61]:
data['periodeSemaine'] = data['DoA'].dt.dayofweek.map(id2str).map(periodeSemaine)
data.head()

,Age,Convicted,DoA,periodeSemaine
0,77,Yes,2014-07-16 14:59:18,millieu
1,28,Yes,2002-12-19 22:17:59,millieu
2,21,No,1994-06-28 09:31:40,debut
3,45,Yes,1985-05-14 09:52:17,debut
4,39,Yes,1985-05-25 14:06:28,week-end


## Gestion des manquants

Il faudra supprimer des manquants. Pour ne pas perdre d'information, il faut insérer une collone comportant l'info de la présence de manquant

In [62]:
data = pd.read_csv("https://bit.ly/missing-values")

data_miss = data.copy()
data_miss.head()

,color,rating
0,NaN,1.16
1,Red,NaN
2,Blue,2.54
3,Red,1.51
4,Red,NaN


Marquer la présence des manquants

In [63]:
data_miss['color_missing'] = data_miss['color'].isna()
data_miss.head()

,color,rating,color_missing
0,NaN,1.16,True
1,Red,NaN,False
2,Blue,2.54,False
3,Red,1.51,False
4,Red,NaN,False


In [64]:
data_miss.insert(1, '_missing', data_miss['color'].isna())
data_miss.head()

,color,_missing,rating,color_missing
0,NaN,True,1.16,True
1,Red,False,NaN,False
2,Blue,False,2.54,False
3,Red,False,1.51,False
4,Red,False,NaN,False


In [65]:
data_miss = data.copy()

# N'inclure que le nom des colonnes NUMERIQUES ou ORDINALES

for nom_colonne in ['color', 'rating'] :
    index_colonne = data_miss.columns.get_loc(nom_colonne)
    data_miss.insert(index_colonne + 1,
                    nom_colonne + '_missing',
                    data_miss[nom_colonne].isna().astype('int8'))
data_miss.head()

,color,color_missing,rating,rating_missing
0,NaN,1,1.16,0
1,Red,0,NaN,1
2,Blue,0,2.54,0
3,Red,0,1.51,0
4,Red,0,NaN,1


 Pour colonnes ordinales, colonnes.cat.codes <br>
 Pour colonnes numériques, astype('int') ou  astype('float')

<strong> Remplacer les valeurs manquates

In [66]:
data_miss['rating'].fillna(data_miss['rating'].mean()) #récupérer et utiliser moyenne

0      1.160000
1      3.066672
2      2.540000
3      1.510000
4      3.066672
         ...   
912    3.400000
913    3.066672
914    1.240000
915    2.640000
916    3.066672
Name: rating, Length: 917, dtype: float64

In [67]:
data_miss['color'].fillna('Missing') #Utiliser une valeur hors plage

0      Missing
1          Red
2         Blue
3          Red
4          Red
        ...   
912       Blue
913        Red
914    Missing
915     Yellow
916       Blue
Name: color, Length: 917, dtype: object

<strong> Traiter la présence de valeurs manquantes [CATE]

S'applique à des variables catégorielles non ordonnées

In [68]:
data_miss['rating'].fillna(data_miss['rating'].mean(), inplace = True)
data_miss.head()

,color,color_missing,rating,rating_missing
0,NaN,1,1.160000,0
1,Red,0,3.066672,1
2,Blue,0,2.540000,0
3,Red,0,1.510000,0
4,Red,0,3.066672,1


In [69]:
data_miss['color'] = data_miss['color'].astype('category')

In [70]:
pd.get_dummies(data_miss,
              dummy_na = True,
              columns = ['color']).head()

,color_missing,rating,rating_missing,color_Blue,color_Orange,color_Red,color_Yellow,color_nan
0,1,1.160000,0,0,0,0,0,1
1,0,3.066672,1,0,0,1,0,0
2,0,2.540000,0,1,0,0,0,0
3,0,1.510000,0,0,0,1,0,0
4,0,3.066672,1,0,0,1,0,0


<strong> Supprimer les lignes comportant des NaN

In [71]:
data = pd.read_csv('https://bit.ly/missing-values-toy')
data

,Colonne Importante,Colonne Inutile,Colonne Peu Utile
0,Oui,NaN,2.0
1,Non,1.0,NaN
2,Légerement,NaN,3.0
3,NaN,1.0,NaN
4,Oui,1.0,4.0
5,NaN,NaN,3.0
6,NaN,2.0,NaN


In [72]:
data.dropna()

,Colonne Importante,Colonne Inutile,Colonne Peu Utile
4,Oui,1.0,4.0


<strong> Supprimer les lignes comportant AU MOINS une valeur manquante

In [73]:
data_any = data.dropna(how = 'any', subset = ['Colonne Importante'])
data_any

,Colonne Importante,Colonne Inutile,Colonne Peu Utile
0,Oui,NaN,2.0
1,Non,1.0,NaN
2,Légerement,NaN,3.0
4,Oui,1.0,4.0


<strong>Supprimer les lignes ne comportant QUE des valeurs manquantes

In [74]:
data_all= data.dropna(how = 'all', subset = ['Colonne Importante', 'Colonne Peu Utile'])
data_all

,Colonne Importante,Colonne Inutile,Colonne Peu Utile
0,Oui,NaN,2.0
1,Non,1.0,NaN
2,Légerement,NaN,3.0
4,Oui,1.0,4.0
5,NaN,NaN,3.0


## Gestion doublons

In [75]:
data=pd.read_csv('https://bit.ly/tx-data')
data.head()

,transaction_id,first_name,last_name,ip_address,amount
0,50563-403,Jacobo,Haresign,18.144.134.48,$647.77
1,54569-3911,Christy,Derl,254.107.123.242,$460.47
2,59667-0024,Florie,Prewer,148.80.11.233,$565.38
3,60429-123,Danika,Shimuk,233.91.221.158,$794.76
4,0187-0771,Annelise,Antonescu,238.143.41.105,$790.41


In [76]:
data.duplicated(keep='first') #marque en TRUE les valeurs déja rencontrées


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15     True
16    False
17    False
18    False
19     True
20     True
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28     True
29    False
30    False
31    False
32     True
33    False
34    False
35    False
36    False
37     True
38    False
39    False
40    False
41    False
42     True
43    False
44    False
45    False
46    False
47    False
dtype: bool

In [77]:
data.duplicated(keep = 'last') # Marque en TRUE la première occurence des valeurs en double

0     False
1     False
2      True
3     False
4     False
5      True
6     False
7      True
8     False
9     False
10    False
11    False
12     True
13    False
14     True
15    False
16     True
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32     True
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
dtype: bool

In [78]:
data.duplicated(keep = False) # Marque en TRUE toutes les lignes posssédant des doublons

0     False
1     False
2      True
3     False
4     False
5      True
6     False
7      True
8     False
9     False
10    False
11    False
12     True
13    False
14     True
15     True
16     True
17    False
18    False
19     True
20     True
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28     True
29    False
30    False
31    False
32     True
33    False
34    False
35    False
36    False
37     True
38    False
39    False
40    False
41    False
42     True
43    False
44    False
45    False
46    False
47    False
dtype: bool

In [79]:
duplicatas = data.duplicated(keep=False,
                                 subset=['transaction_id']) #Montre toutes les lignes dupliquées
data[duplicatas].sort_values(['transaction_id']).head() #regroupe les doublons

,transaction_id,first_name,last_name,ip_address,amount
16,0066-0508,Jedd,Hartman,21.55.247.91,$749.96
42,0066-0508,Jedd,Hartman,21.55.247.91,$749.96
5,53942-299,Upton,Emig,103.190.123.125,$759.53
19,53942-299,Upton,Emig,103.190.123.125,$759.53
2,59667-0024,Florie,Prewer,148.80.11.233,$565.38


In [80]:

data_unique=data.drop_duplicates(subset=['transaction_id'])
data_unique.head(10)

,transaction_id,first_name,last_name,ip_address,amount
0,50563-403,Jacobo,Haresign,18.144.134.48,$647.77
1,54569-3911,Christy,Derl,254.107.123.242,$460.47
2,59667-0024,Florie,Prewer,148.80.11.233,$565.38
3,60429-123,Danika,Shimuk,233.91.221.158,$794.76
4,0187-0771,Annelise,Antonescu,238.143.41.105,$790.41
5,53942-299,Upton,Emig,103.190.123.125,$759.53
6,41163-519,Rachael,Housley,3.188.252.248,$871.72
7,60289-247,Corri,Rockcliffe,46.150.208.18,$5.68
8,0527-1742,Ambros,Goulding,174.124.224.39,$726.12
9,57237-083,Roma,Addekin,37.14.63.174,$526.89


## Transformer des données
Transformer un jeu de données en données ordinales

In [81]:

data=pd.read_csv("https://bit.ly/felonies-dataset")
data.head()

,Date of Arrest,Age,Convicted
0,2014-07-16 14:59:18,77,Yes
1,2002-12-19 22:17:59,28,Yes
2,1994-06-28 09:31:40,21,No
3,1985-05-14 09:52:17,45,Yes
4,1985-05-25 14:06:28,39,Yes


In [82]:
data['Age'].describe()


count    1000.000000
mean       54.605000
std        25.225172
min        10.000000
25%        33.000000
50%        53.000000
75%        76.000000
max        99.000000
Name: Age, dtype: float64

In [83]:
o=pd.cut(data['Age'],
      bins=[10,14,20,65,100],
      labels=['Enfant','Ado','Adult','Senior'])
o

0      Senior
1       Adult
2       Adult
3       Adult
4       Adult
        ...  
995    Senior
996    Senior
997    Senior
998     Adult
999     Adult
Name: Age, Length: 1000, dtype: category
Categories (4, object): ['Enfant' < 'Ado' < 'Adult' < 'Senior']

In [84]:
o.isna().sum()


13

Des cas n'ont pas été pris en compte dans le choix des bornes ce qui a créé des NaN.



## Transformer les données en catégories hiérarchisées [ORDINAL]¶


In [85]:
ordre=['debut','milieu','week-end']
data['periodeSemaine']=(data['periodeSemaine'].astype('category').
                        cat.reorder_categories(ordre).
                        cat.as_ordered())
data['periodeSemaine'].dtype

KeyError: 'periodeSemaine'

In [86]:
data.head()

,Date of Arrest,Age,Convicted
0,2014-07-16 14:59:18,77,Yes
1,2002-12-19 22:17:59,28,Yes
2,1994-06-28 09:31:40,21,No
3,1985-05-14 09:52:17,45,Yes
4,1985-05-25 14:06:28,39,Yes


### Convertir la colonne "category" en "int" [ORDINAL]¶


In [87]:
data['periodeSemaine']=data['periodeSemaine'].cat.codes
data.head()

KeyError: 'periodeSemaine'

Bonus : Sauvegarder des objets
Méthode intéressante pour sauvegarder le vocabulaire et pouvoir le réutiliser

A Savoir : On ne peut pas pickle des objets numpy

In [88]:
import pickle
path=('C:/Users/elisa/Desktop/Python H3/Devoir Pandas/dictionnaire.pkl')

#en écriture
with open (path, 'wb') as f:
    pickle.dump(id2str,f) #pickle.dump(object, directory)
    
#en lecture
with open (path, 'rb') as f:
    objet=pickle.load(f) #pickle.dump(object, directory)
objet

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/elisa/Desktop/Python H3/Devoir Pandas/dictionnaire.pkl'

### Sauvegarder/Exporter une base de données¶


In [89]:
data.to_csv('C:/Users/elisa/Desktop/Python H3/Devoir Pandas/dataset.csv',
           index=None)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/elisa/Desktop/Python H3/Devoir Pandas/dataset.csv'